In [16]:
import pandas as pd
import numpy as np
from dataset import Dataset
import warnings

# SettingWithCopyWarning 무시
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


ds = Dataset()
JSON_KEY_PATH = "./sprintda05-soomin.json"

In [17]:
master_table = pd.read_csv('./dataset/master-v1.csv')
point_table = pd.read_csv('./dataset/use_point_within_signup_N_days.csv')
candidate_table = pd.read_csv('./dataset/candidate_chosen_ratio.csv')

In [18]:
master_table = master_table.rename(columns={'votes_within_3d':'votes_within_3d_count','chosen_user_count':'vote_nunique_user_count'})

In [32]:
master_table

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,ret_group,ret_group_over,ret_group_under,gender,group_id,school_id,grade,votes_within_3d_count,reported_user_count,vote_nunique_user_count,period_friend_count
0,838023,22,5,2023-04-29,2023-05-04,5,2023-04-29,0.515152,50 이상,50 이상,NaN,F,233.0,271.0,1.0,7,0.0,11.0,0.0
1,838466,127,6,2023-05-02,2023-05-07,5,2023-05-02,0.569444,50 이상,50 이상,NaN,F,321.0,369.0,2.0,77,0.0,28.0,63.0
2,838642,1,1,2023-04-29,2023-04-29,0,2023-04-29,0.515152,50 이상,50 이상,NaN,F,356.0,271.0,2.0,1,0.0,1.0,5.0
3,839357,205,8,2023-04-29,2023-05-07,8,2023-04-29,0.515152,50 이상,50 이상,NaN,M,519.0,271.0,1.0,114,0.0,45.0,52.0
4,840293,170,7,2023-05-03,2023-05-15,12,2023-05-03,0.422977,50 미만,NaN,50 미만,F,838.0,369.0,1.0,130,0.0,32.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,1510561,9,1,2023-05-25,2023-05-25,0,2023-05-25,0.451613,50 미만,NaN,50 미만,M,73885.0,4426.0,2.0,68,0.0,0.0,0.0
4699,1511957,8,1,2023-05-25,2023-05-25,0,2023-05-25,0.451613,50 미만,NaN,50 미만,F,29558.0,5520.0,2.0,23,0.0,0.0,0.0
4700,1512422,22,1,2023-05-25,2023-05-25,0,2023-05-25,0.451613,50 미만,NaN,50 미만,F,72313.0,4426.0,2.0,127,0.0,0.0,0.0
4701,1512542,7,1,2023-05-25,2023-05-25,0,2023-05-25,0.451613,50 미만,NaN,50 미만,F,72313.0,4426.0,2.0,167,0.0,0.0,0.0


In [19]:
def table_merging(master_table, merge_table):
    result = master_table.merge(
    merge_table,
    on = 'user_id',
    how='left'
).copy()
    return result

## 가입 후 n일 이내 결제 유무

In [20]:
# 가입 후 7일 이내 결제 유무
date_user_table = point_table[['user_id','sign_up_date','first_vote_date','first_point_use_date','first_point_use_amount','vote_sameday_signup']]
date_user_table['sign_up_date'] = pd.to_datetime(date_user_table['sign_up_date'])
date_user_table['first_vote_date'] = pd.to_datetime(date_user_table['first_vote_date'])
date_user_table['first_point_use_date'] = pd.to_datetime(date_user_table['first_point_use_date'])

# 조건 컬럼
date_user_table['days_to_vote'] = (date_user_table['first_vote_date'] - date_user_table['sign_up_date']).dt.days
# 조건 컬럼
date_user_table['days_to_point'] = (date_user_table['first_point_use_date'] - date_user_table['sign_up_date']).dt.days
# 조건 컬럼
date_user_table['days_to_point_after_vote'] = (date_user_table['first_point_use_date'] - date_user_table['first_vote_date']).dt.days

final_master_table = table_merging(master_table, date_user_table)

## 처음 투표한 시간

In [21]:
# 처음 투표한 시간
user_table = master_table[['user_id']].copy()
user_table = user_table.merge(point_table[['user_id','first_vote_date']], on='user_id',how='left')
user_table['first_vote_hour'] = pd.to_datetime(user_table['first_vote_date']).dt.hour

final_master_table = table_merging(final_master_table, user_table[['user_id','first_vote_hour']])

## 투표 첫 날 투표 건수

In [22]:
qusetion_table = ds.votes.accounts_userquestionrecord()[['user_id','created_at','id']]
qusetion_table['day'] = pd.to_datetime(qusetion_table['created_at']).dt.floor("d")
qusetion_table = qusetion_table.drop(columns=['created_at'])

user_table['day'] = pd.to_datetime(user_table['first_vote_date']).dt.floor('d')

merged_table =pd.merge(
    user_table[['user_id','day']],
    qusetion_table,
    on = ['user_id','day'],
    how = 'left'
)

merged_table['first_voteday_question_count'] = (
    merged_table
    .groupby(['user_id', 'day'])['user_id']
    .transform('count')
)

merged_table = (
    merged_table
    .drop_duplicates(subset='user_id')
    .drop(columns=['id'])
)

final_master_table = table_merging(final_master_table, merged_table.drop(columns='day'))

## 열람 건 수 (비율)

In [ ]:
record = (
    ds.votes.accounts_userquestionrecord()
    .query('created_at < "2023-08-01"')
    .query('created_at >= "2023-04-28"')
)

has_read_table = record[['chosen_user_id','has_read','id']].copy()

has_read_table['total_voted'] =  (
    has_read_table
    .groupby(['chosen_user_id'])['id']
    .transform('count')
)
has_read_table['total_read'] = (
    has_read_table
    .groupby(['chosen_user_id'])['has_read']
    .transform('sum')
)
has_read_table = has_read_table[['chosen_user_id','total_voted','total_read']].rename(columns={'chosen_user_id':'user_id'}).drop_duplicates(subset='user_id')

fianl_master_table = table_merging(final_master_table, has_read_table)

## 투표 UNIQUE 유저 선택 수  (투표 대상 수)

In [24]:
# master-v1 통합

## 친구 수

In [25]:
# master-v1 통합

## 이성에게 받은 투표 수 (날짜 수정 필요)

In [26]:
opposite_sex_votes = pd.read_csv('./dataset/master_with_vote_stats.csv')
opposite_sex_votes = opposite_sex_votes.rename(columns={'opposite_votes_received':'opposite_sex_votes_received_count','opposite_vote_ratio':'opposite_sex_vote_received_ratio'})
final_master_table = table_merging(fianl_master_table,opposite_sex_votes.drop(columns=['gender','total_votes_received']))

## 첫 질문 유형

In [27]:
question_category = pd.read_csv('./dataset/first_question_category.txt')
fianl_master_table = table_merging(final_master_table,question_category.drop(columns='created_at'))

## 소속학교, 학년 성별

In [17]:
# Master Table 에 내용 있음

## 투표 등장 대비 선택받은 비율

In [29]:
# 투표 등장 대비 선택받은 비율
final_master_table = table_merging(final_master_table, candidate_table.drop(columns='chosen_user_id'))

# 최종본

In [3]:
import pandas as pd

In [11]:
final_master_table.to_csv('./integrated_master_table.csv', encoding='UTF-8', index=False)